In [171]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [172]:
import warnings
warnings.filterwarnings('ignore')


import scipy as sp
import pandas as pd
import re
import numpy as np


# Data visualization
import matplotlib as mlt
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Scalers
from sklearn.preprocessing import StandardScaler

#discritization
from sklearn.preprocessing import LabelEncoder

#Models
from sklearn.ensemble import RandomForestClassifier

#evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [173]:
train = pd.read_csv (r'../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv (r'../input/house-prices-advanced-regression-techniques/test.csv')

#describe train dataset
train.head(10).style.background_gradient()
print('----------------------------TRAIN----------------------------------\n')
print("Total number of features: \n", len(train.columns)-1)
categorical = train.dtypes[train.dtypes == 'object'].index
print('Categorical features: \n',categorical,"\nNumber of features: \n", len(categorical))
numerical = train.dtypes[train.dtypes != 'object'].index
print('Numerical features: \n',numerical, "\nNumber of features: \n",len(numerical)-1)

#describe test dataset
test.head(10).style.background_gradient()
print('----------------------------TEST----------------------------------\n')
print("Total number of features: \n", len(test.columns))
categorical = test.dtypes[test.dtypes == 'object'].index
print('Categorical features: \n',categorical,"\nNumber of features: \n", len(categorical))
numerical = test.dtypes[test.dtypes != 'object'].index
print('Numerical features: \n',numerical, "\nNumber of features: \n",len(numerical))

#handle missing data
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    types = [str(data[i].dtype) for i in data.columns]
    df = pd.DataFrame({'Total':total, 'Precent':percent, 'Types':types})
    return(df)
df_missing_train = missing_data(train)

#select feature with missing values
df_missing_train[(df_missing_train.select_dtypes(include=['number']) != 0).any(1)]

In [174]:
#check distribution of all continuous features
train_data_num = train.drop(columns ='Id').select_dtypes(exclude=['object'])
train_data_num.hist(figsize=(25, 30), bins=30)

#disctribution of saleprice
SalePrice = train['SalePrice']
plot =sns.histplot(x=SalePrice, kde =True, multiple ='stack',bins=30)
sns.set(rc = {'figure.figsize':(20,8)})

In [175]:
#basic missing value mannual imputation

replaced_data_values = dict({
    "Alley":"No_alley_acces",
    "BsmtQual":"No_Basement",
    "BsmtCond":"No_Basement",
    "BsmtExposure": "No_Basement",
   "BsmtFinType1": "No_Basement",
   "BsmtFinType2": "No_Basement",
   "FireplaceQu": "No_Fireplace",
   "GarageType": "No_Garage",
   "GarageFinish": "No_Garage",
   "GarageCond": "No_Garage",
   "GarageQual": "No_Garage",
   "PoolQC": "No_Pool",
   "MiscFeature": "Nothing",
   "Fence": "No_Fence",
   "MasVnrType": "Nothing"
})

def replace_data(df):
    for idx in replaced_data_values:
        df[idx].fillna(replaced_data_values[idx], inplace=True)
    return df

train1 = replace_data(train)
test1 = replace_data(test)
df_combined = pd.concat([train1.drop('SalePrice', axis=1), test1], axis=0)
#missing value imputation for categorical&continuous features
for i in categorical:
    df_combined[i].fillna(train1[i].mode(), inplace=True)
for i in numerical:
    df_combined[i].fillna(train1[i].median(), inplace=True)
    
missing_df= missing_data(df_combined)
missing_df[(missing_df.select_dtypes(include=['number']) != 0).any(1)]

In [176]:
#handle left nan values
df_combined['Utilities'].unique()
for i in categorical:
    df_combined[i].replace(np.nan, 0, inplace=True)

In [177]:
df_combined = df_combined.drop(columns ='Id')
X_train = pd.get_dummies(df_combined)[:len(train1)]
X_test = pd.get_dummies(df_combined)[len(train1):]

In [178]:
#check correlation between features&y: correlation matrix
corrmat = train.corr()
plot = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);
#check the 10 largest variable that are correlated with 'Saleprice'
k = 15
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index # selected feautre with hightest corr
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()
print(cols)


#keep columns has high correlation
columns_to_maintain = []
for i in X_train.columns:
    if abs(X_train[i].corr(train.SalePrice))>0.40:
        columns_to_maintain.append(i)
columns_to_maintain

In [179]:
X_train

In [180]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X = X_train
y= SalePrice
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=12)
model1 = LinearRegression()
model1.fit(X_train, y_train)
print(model1.score(X_val, y_val))

In [181]:
from sklearn.ensemble import RandomForestRegressor
model2 = RandomForestRegressor()
model2.fit(X_train, y_train)
print(model2.score(X_val, y_val))

In [182]:
predictions = model2.predict(X_test)
pred = pd.DataFrame(predictions)
pred.rename(columns={0: 'SalePrice'}, inplace=True)
final = pd.concat([test.Id, pred], axis=1)
final.head()

In [183]:
final.to_csv('./submission.csv', index=False)